In [44]:
import pandas as pd
from datahandler import DataHandler
from db import Database

In [45]:
# 1. Python: Download a timeseries of daily deaths per country
url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"
covid_deaths_df = pd.read_csv(url)
covid_deaths_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259 entries, 0 to 258
Data columns (total 78 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Province/State  81 non-null     object 
 1   Country/Region  259 non-null    object 
 2   Lat             259 non-null    float64
 3   Long            259 non-null    float64
 4   1/22/20         259 non-null    int64  
 5   1/23/20         259 non-null    int64  
 6   1/24/20         259 non-null    int64  
 7   1/25/20         259 non-null    int64  
 8   1/26/20         259 non-null    int64  
 9   1/27/20         259 non-null    int64  
 10  1/28/20         259 non-null    int64  
 11  1/29/20         259 non-null    int64  
 12  1/30/20         259 non-null    int64  
 13  1/31/20         259 non-null    int64  
 14  2/1/20          259 non-null    int64  
 15  2/2/20          259 non-null    int64  
 16  2/3/20          259 non-null    int64  
 17  2/4/20          259 non-null    int

In [46]:
# 2. Python: Convert the table so that each country and each day is a separate row 
data = DataHandler()
# Implementation details are in datahandler.py
total_deaths_df = data.get_total_deaths_per_country_and_day(covid_deaths_df)
total_deaths_df.info()
total_deaths_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13394 entries, 0 to 13393
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   country  13394 non-null  object        
 1   date     13394 non-null  datetime64[ns]
 2   deaths   13394 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 314.0+ KB


,country,date,deaths
0,Afghanistan,2020-01-22,0
1,Albania,2020-01-22,0
2,Algeria,2020-01-22,0
3,Andorra,2020-01-22,0
4,Angola,2020-01-22,0


In [47]:
# 3.  Python: Provide code to upload the table from step 3 (assuming 2) into an SQL table named deaths_total

db_name = 'Covid19.db'
db = Database(db_name)

# Implementation details are in db.py
db.create_total_deaths_table() # Creates a table named deaths_total in database
total_rows, new_rows = db.insert_to_deaths_total_table(total_deaths_df)
print('Total rows in deaths_total table: '+str(total_rows))
print('Newly added rows in deaths_total table: '+str(new_rows))

Total rows in deaths_total table: 13394
Newly added rows in deaths_total table: 0


In [48]:
# 4.  Python: From the data in step 2, calculate the daily change in deaths for each country

# Implementation details are in datahandler.py
daily_change_df = data.get_daily_change_in_deaths(total_deaths_df)
daily_change_df.head()

,country,date,deaths_change
0,Afghanistan,2020-01-22,NaN
181,Afghanistan,2020-01-23,0.0
362,Afghanistan,2020-01-24,0.0
543,Afghanistan,2020-01-25,0.0
724,Afghanistan,2020-01-26,0.0


In [49]:
# 5.  Python: Provide code to upload the table from step 4 into an SQL table named deaths_change_python

db.create_deaths_change_python_table()
total_rows, new_rows = db.insert_to_deaths_change_python_table(daily_change_df)
print('Total rows in deaths_change_table: ' +str(total_rows))
print('Newly added rows in deaths_change_table: '+str(new_rows))


Total rows in deaths_change_table: 13394
Newly added rows in deaths_change_table: 0


In [50]:
# 6. SQL: Provide SQL code to calculate the daily change for each country using only the data from deaths_total and save it into an SQL table named deaths_change_sql

db.execute_query("DROP TABLE IF EXISTS deaths_change_sql;")

deaths_change_sql = "CREATE TABLE deaths_change_sql AS \
                        SELECT country, date, \
                        deaths - LAG(deaths) OVER (PARTITION BY country ORDER BY date) AS deaths_change \
                        FROM deaths_total;"
db.execute_query(deaths_change_sql)

# Some verifications
tables = db.execute_query("SELECT name FROM sqlite_master WHERE type ='table' AND name NOT LIKE 'sqlite_%';")
print(tables)

with db.create_connection() as con:
    result = pd.read_sql("SELECT * FROM deaths_change_sql", con=con)

# Do some structural only changes to daily_changes_df 
daily_change_df.sort_values(['country','date'], inplace = True)
daily_change_df.reset_index(drop=True, inplace=True)
daily_change_df['date'] = daily_change_df['date'].dt.strftime('%Y-%m-%d %H:%M:%S')
if daily_change_df.equals(result):
    print("Both Python and SQL codes produce identical results")
    
result

[('deaths_total',), ('deaths_change_python',), ('deaths_change_sql',)]
Both Python and SQL codes produce identical results


,country,date,deaths_change
0,Afghanistan,2020-01-22 00:00:00,NaN
1,Afghanistan,2020-01-23 00:00:00,0.0
2,Afghanistan,2020-01-24 00:00:00,0.0
3,Afghanistan,2020-01-25 00:00:00,0.0
4,Afghanistan,2020-01-26 00:00:00,0.0
...,...,...,...
13389,Zimbabwe,2020-03-31 00:00:00,0.0
13390,Zimbabwe,2020-04-01 00:00:00,0.0
13391,Zimbabwe,2020-04-02 00:00:00,0.0
13392,Zimbabwe,2020-04-03 00:00:00,0.0
